# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 2: geotag
-----
###### Student Name(s):Xiaoxuan Li 933206
###### Python version:Python3

In [1]:
#import classifiers
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

#import evaluate tools
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import time
import csv

In [ ]:
#X, y = [], []
#with open('train-raw.tsv') as trainData:
    #tsvreader = csv.reader(trainData, delimiter = "\t")
    #for line in tsvreader:
        #y.append(line[1])
        #X.append(line[2])
    #y = y[1:]
    #X = X[1:]
    
    #vectoriser = CountVectorizer()
    #X = vectoriser.fit_transform(X)
    

#mi = SelectKBest(score_func=mutual_info_classif, k=1000)
#mi.fit(X, y)

#for feat_num in mi.get_support(indices=True):
    #print(vectoriser.get_feature_names()[feat_num])
    
#X_mi = mi.transform(X)

   

In [3]:
#should appear knn
models = [GaussianNB(),
          MultinomialNB(),
          DecisionTreeClassifier(max_depth=None, criterion="entropy"),
          svm.LinearSVC(),
          svm.SVC(kernel='rbf', gamma=0.7, C=1.0),
          LogisticRegression(),
          DummyClassifier(strategy='most_frequent'),
          DecisionTreeClassifier(max_depth=1, criterion="entropy")]
titles = ['GNB',
          'MNB',
          'Decision Tree',
          'LinearSVC',
          'SVC with rbf kernel',
          'Logistic Regression',
          'Zero-R'
          'One-R']

In [3]:
X0, X, y = [], [], []
with open('train-top100.csv') as trainData:
    tsvreader = csv.reader(trainData, delimiter = ",")
    for line in tsvreader:
        y.append(line[-1])
        X0.append(line[1:-1])
    y = y[1:]
    X0 = X0[1:]
    
    for entry in X0:
        X.append(np.array(entry).astype(np.float))
    
    #vectoriser = CountVectorizer()
    #X = vectoriser.fit_transform(X)
    

In [ ]:
for title, model in zip(titles, models):
    start = time.time()
    model.fit(X, y)
    acc = np.mean(cross_val_score(model, X, y, cv=10))
    end = time.time()
    t = end - start
    print(title, acc, 'time:', t)
    

GNB 0.3348166405674557 time: 26.21943974494934
MNB 0.33661609757412075 time: 7.445412874221802
Decision Tree 0.33510699304154123 time: 1024.3456909656525


/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the num

LinearSVC 0.33798018680647457 time: 517.5005738735199


In [2]:
models = [LogisticRegression(),
          DummyClassifier(strategy='most_frequent'),
          DecisionTreeClassifier(max_depth=1, criterion="entropy")]
titles = ['Logistic Regression',
          'Zero-R'
          'One-R']

In [4]:
for title, model in zip(titles, models):
    start = time.time()
    model.fit(X, y)
    acc = np.mean(cross_val_score(model, X, y, cv=10))
    end = time.time()
    t = end - start
    print(title, acc, 'time:', t)

/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/linea

Logistic Regression 0.3370030383373954 time: 29.81513500213623
Zero-ROne-R 0.25 time: 1.2296059131622314


In [10]:
np.random.seed(1)

class StackingClassifier():

    def __init__(self, classifiers, metaclassifier):
        self.classifiers = classifiers
        self.metaclassifier = metaclassifier

    def fit(self, X, y):
        for clf in self.classifiers:
            clf.fit(X, y)
        X_meta = self._predict_base(X)
        self.metaclassifier.fit(X_meta, y)
    
    def _predict_base(self, X):
        yhats = []
        for clf in self.classifiers:
            yhat = clf.predict_proba(X)
            yhats.append(yhat)
        yhats = np.concatenate(yhats, axis=1)
        #print(yhats.shape)
        #assert yhats.shape[0] == X.shape[0]
        assert len(yhats) == len(X)
        return yhats
    
    def predict(self, X):
        X_meta = self._predict_base(X)
        yhat = self.metaclassifier.predict(X_meta)
        return yhat
    def score(self, X, y):
        yhat = self.predict(X)
        return accuracy_score(y, yhat)
    


classifiers = [LogisticRegression(),
                GaussianNB(),
                MultinomialNB(),
                DecisionTreeClassifier()]

meta_classifier = DecisionTreeClassifier()
stacker = StackingClassifier(classifiers, meta_classifier)

def load_data(file_name):
    X = []
    X0 = []
    y = []
    with open(file_name, mode='r') as fin:
        for line in fin:
            atts = line.strip().split(",")
            X0.append(atts[1:-1]) #all atts minus the last one
            y.append(atts[-1])
    y = y[1:]
    X0 = X0[1:]
    for entry in X0:
        X.append(np.array(entry).astype(np.float))
    return X, y

X, y = load_data('train-top100.csv')
print('labels:', set(y))
stacker.fit(X, y)
X_test, y_test = load_data('dev-top100.csv')
#acc = np.mean(cross_val_score(stacker, X, y, cv=10))
print('stacker acc:', stacker.score(X_test, y_test))


labels: {'Melbourne', 'Sydney', 'Perth', 'Brisbane'}


/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/valerie/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


stacker acc: 0.3102422553328331


In [25]:
#generate the final test output
X = []
X0 = []
X1 = []
y = []
with open("test-top100.csv", mode='r') as fin:
    for line in fin:
        atts = line.strip().split(",")
        X0.append(atts[1:-1])
        y.append(atts[0])
    X0 = X0[1:]
    y= y[1:]
    for entry in X0:
        X1.append(np.array(entry).astype(np.float))
        
    yhat = stacker.predict(X1)

count = 0
for entry in yhat:
    X.append(entry)
    if(entry == "Brisbane"):
        count = count +1
print(len(X))
print(count)


108148
95735


In [32]:
f_out = open("result.csv",'w')
f_out.write("Id")
f_out.write(",")
f_out.write("Class")
f_out.write("\n")
for entryid,entry in zip(y, yhat):
    f_out.write(entryid)
    f_out.write(",")
    f_out.write(entry)
    f_out.write("\n")

f_out.close()
    
    

In [ ]:
#experiment on dev data set
X, y = [], []
with open('dev-raw.tsv') as devData:
    tsvreader = csv.reader(devData, delimiter = "\t")
    for line in tsvreader:
        if(len(line)== 3):
            y.append(line[1])
            X.append(line[2])
    
    
    vectoriser = CountVectorizer()
    X = vectoriser.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [ ]:
models = [GaussianNB(),
          MultinomialNB()]
          #DecisionTreeClassifier(max_depth=None, criterion="entropy"),
          #svm.LinearSVC(),
          #LogisticRegression()]
titles = ['GNB',
          'MNB']
          #'Decision Tree',
          #'LinearSVC',
          #'Logistic Regression']


In [ ]:
#feature engineer(choose from x2/mi repsentation)
k = 10000

x2 = SelectKBest(chi2, k=k)
x2.fit(X_train,y_train)

#for feat_num in x2.get_support(indices=True):
    #print(vectoriser.get_feature_names()[feat_num])
    
X_train_x2 = x2.transform(X_train)
X_test_x2 = x2.transform(X_test)



mi = SelectKBest(score_func=mutual_info_classif, k=k)
mi.fit(X_train,y_train)

#for feat_num in mi.get_support(indices=True):
    #print(vectoriser.get_feature_names()[feat_num])
    
X_train_mi = mi.transform(X_train)
X_test_mi = mi.transform(X_test)


Xs = [(X_train, X_test), (X_train_x2, X_test_x2), (X_train_mi, X_test_mi)]
X_names = ['complete', 'x2', 'mi']
for title, model in zip(titles, models):
    for X_name, X in zip(X_names, Xs):
        X_train_t, X_test_t = X
        model.fit(X_train_t.todense(), y_train)
        acc = model.score(X_test_t.todense(), y_test)
        print('k', k, title, 'features', X_name, 'acc',  acc)

In [ ]:
#tune hyperparams using dev(choose the number of k in kNN)


